In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
%matplotlib inline
from IPython.display import SVG, Image
from keras.utils.vis_utils import model_to_dot

import re
import itertools
import random

import numpy as np
import scipy
import pandas as pd
import gensim

from sklearn import datasets
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn import cluster, datasets, mixture
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import decomposition

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

from keras.utils import to_categorical

In [ ]:
def hexbin(x, y, color, **kwargs):
    cmap = sns.light_palette(color, as_cmap=True)
    plt.hexbin(x, y, cmap=cmap, **kwargs)
def scatter(x, y, color, **kwargs):
    plt.scatter(x, y, marker='.')

In [ ]:
X_df = pd.read_csv('../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv', skiprows=[1])
X_df.head()

In [ ]:
# tmp = X_df['Q1'].values
# tmp[tmp=='Prefer to self-describe'] = 'other'
# tmp[tmp=='Prefer not to say'] = 'other'

In [ ]:
# enc = preprocessing.OneHotEncoder()

In [ ]:
# enc.fit(tmp.reshape((-1,1)))

In [ ]:
# enc.categories_

In [ ]:
# enc.transform(tmp.reshape((-1,1))).shape

In [ ]:
def transform(x, col, other=[]):
    tmp = x[col].copy().values
    for ee in other:
        tmp[tmp==ee] = 'zzz'
    enc = preprocessing.OneHotEncoder()
    enc.fit(tmp.reshape((-1,1)))
    res0 = enc.transform(tmp.reshape((-1,1))).astype(int)
    cols = [col+'_'+str(ee) for ee in enc.categories_[0]]
    print(cols)
    res = pd.DataFrame(res0.todense(), columns=cols)
    return enc, res

In [ ]:
enc, resQ1 = transform(X_df, 'Q1', other=[])
print(enc, enc.categories_, resQ1.shape)
resQ1.head()

In [ ]:
enc, resQ2 = transform(X_df, 'Q2', other=['Prefer to self-describe', 'Prefer not to say'])
print(enc, enc.categories_, resQ2.shape)
resQ2.head()

In [ ]:
enc, resQ3 = transform(X_df, 'Q3', other=['I do not wish to disclose my location', 'Other'])
print(enc, enc.categories_, resQ3.shape)
resQ3.head()

In [ ]:
enc, resQ4 = transform(X_df, 'Q4', other=['I prefer not to answer', 'Other'])
print(enc, enc.categories_, resQ4.shape)
resQ4.head()

In [ ]:
enc, resQ5 = transform(X_df, 'Q5', other=['Other'])
print(enc, enc.categories_, resQ5.shape)
resQ5.head()

In [ ]:
enc, resQ6 = transform(X_df, 'Q6', other=[])
print(enc, enc.categories_, resQ6.shape)
resQ6.head()

In [ ]:
def transform_MA(x, col, other=[]):
    idx_col = [True if re.match(col+'_Part_[0-9]+$', ee) else False for ee in x.columns.values]
    #idx_col = [True if re.search('^Q11_Part_[0-9]+$', ee) else False for ee in x.columns.values]
    idx_col = x.columns.values[idx_col]
    print(idx_col)
    df_list = []
    for icol in idx_col:
        enc, res0 = transform(x, icol, other=other)
        res_cols = res0.columns.values
        res_cols_drop = res_cols[[True if re.search('_nan$', ee) else False for ee in res_cols]]
        res0 = res0.drop(res_cols_drop, axis=1)
        df_list.append(res0)
    res = pd.concat(df_list, axis=1)
    return res

In [ ]:
resQ7 = transform_MA(X_df, 'Q7', other=[])
resQ7.head()

In [ ]:
enc, resQ8 = transform(X_df, 'Q8', other=[])
print(enc, enc.categories_, resQ8.shape)
resQ8.head()

In [ ]:
resQ9 = transform_MA(X_df, 'Q9', other=[])
resQ9.head()

In [ ]:
resQ10 = transform_MA(X_df, 'Q10', other=[])
resQ10.head()

In [ ]:
res_all = pd.concat(
    [
        resQ1,
        resQ2,
        resQ3,
        resQ4,
        resQ5,
        resQ6,
        resQ7,
        resQ8,
        resQ9,
        resQ10,
    ], axis=1)
print(res_all.shape)
res_all.head()

In [ ]:
res_all.to_csv('encoded_01.csv')